In [18]:
import pandas as pd
import numpy as np
from typing import Tuple, Dict, List
import random

In [19]:
csv_data = pd.read_csv('API Output-Pitchers.csv')

# Calculate number of singles
csv_data["singles"] = (
    csv_data["hits"] - csv_data["doubles"] - csv_data["triples"] - csv_data["homeRuns"]
)

# Replace missing or zero hits to avoid division errors
csv_data["hits"] = csv_data["hits"].replace(0, np.nan)

# Calculate each as a percentage of total hits
csv_data["singles_pct"] = (csv_data["singles"] / csv_data["hits"]) * 100
csv_data["doubles_pct"] = (csv_data["doubles"] / csv_data["hits"]) * 100
csv_data["triples_pct"] = (csv_data["triples"] / csv_data["hits"]) * 100
csv_data["homeRuns_pct"] = (csv_data["homeRuns"] / csv_data["hits"]) * 100

# Fill NaN percentages with 0 if player has 0 hits
csv_data= csv_data.fillna(0)

csv_data["TeamName"].unique()



array(['Athletics', 'Pittsburgh Pirates', 'San Diego Padres',
       'Seattle Mariners', 'San Francisco Giants', 'St. Louis Cardinals',
       'Tampa Bay Rays', 'Texas Rangers', 'Toronto Blue Jays',
       'Minnesota Twins', 'Philadelphia Phillies', 'Atlanta Braves',
       'Chicago White Sox', 'Miami Marlins', 'New York Yankees',
       'Milwaukee Brewers', 'Los Angeles Angels', 'Arizona Diamondbacks',
       'Baltimore Orioles', 'Boston Red Sox', 'Chicago Cubs',
       'Cincinnati Reds', 'Cleveland Guardians', 'Colorado Rockies',
       'Detroit Tigers', 'Houston Astros', 'Kansas City Royals',
       'Los Angeles Dodgers', 'Washington Nationals', 'New York Mets'],
      dtype=object)

In [ ]:
import pandas as pd
import numpy as np
import random
from typing import Dict

class PitcherBasedGameSimulator:
    """
    Simulates a complete baseball game using only pitcher statistics.
    All outcomes are determined by the pitcher's performance metrics.
    """

    def __init__(self, home_pitcher: pd.Series, away_pitcher: pd.Series,
             home_team_name: str = "Home", away_team_name: str = "Away",
             debug: bool = False):
        self.home_pitcher = home_pitcher
        self.away_pitcher = away_pitcher
        self.home_team_name = home_team_name
        self.away_team_name = away_team_name
        self.debug = debug

        # Game state
        self.inning = 1
        self.is_top_inning = True
        self.outs = 0
        self.bases = [False, False, False]
        
        # Score tracking
        self.home_score = 0
        self.away_score = 0

        # Pitch count
        self.home_pitcher_pitches = 0
        self.away_pitcher_pitches = 0
        self.home_pitcher_active = True
        self.away_pitcher_active = True

        # Box score
        self.box_score = {'home': {'R': 0, 'H': 0, 'E': 0},
                        'away': {'R': 0, 'H': 0, 'E': 0}}
        self.play_by_play = []

    def normalize_probability(self, value, default=0.0):
        if pd.isna(value):
            return default
        return float(value)

    def get_current_pitcher(self) -> pd.Series:
        if self.is_top_inning:
            
            if self.home_pitcher_active:
                return self.home_pitcher
            else:
                return None
        else:
            if self.away_pitcher_active:
                return self.away_pitcher
            else:
                return None

    def calculate_pitcher_outcome_probabilities(self, pitcher: pd.Series) -> Dict:
        bf = max(self.normalize_probability(pitcher.get('battersFaced', 1)), 1)
        strikeouts = self.normalize_probability(pitcher.get('strikeOuts', 0))
        walks = self.normalize_probability(pitcher.get('baseOnBalls', 0))
        hits = self.normalize_probability(pitcher.get('hits', 0))
        homeruns = self.normalize_probability(pitcher.get('homeRuns', 0))
        hit_batsmen = self.normalize_probability(pitcher.get('hitBatsmen', 0))

        k_rate = strikeouts / bf
        bb_rate = walks / bf
        hbp_rate = hit_batsmen / bf
        hr_rate = homeruns / bf
        hit_rate = hits / bf

        # Estimate hit type distribution
        singles_pct = self.normalize_probability(pitcher.get('singles_pct', 65))
        doubles_pct = self.normalize_probability(pitcher.get('doubles_pct', 20))
        triples_pct = self.normalize_probability(pitcher.get('triples_pct', 2))

        single_rate = hit_rate * singles_pct / 100
        double_rate = hit_rate * doubles_pct / 100
        triple_rate = hit_rate * triples_pct / 100

        # Estimate outs in play
        outs_in_play = bf - (strikeouts + walks + hit_batsmen + hits)
        ground_out_rate = outs_in_play * 0.55 / bf
        air_out_rate = outs_in_play * 0.45 / bf

        return {
            'single': single_rate,
            'double': double_rate,
            'triple': triple_rate,
            'homerun': hr_rate,
            'walk': bb_rate,
            'hbp': hbp_rate,
            'strikeout': k_rate,
            'ground_out': ground_out_rate,
            'air_out': air_out_rate
        }

    def safe_choice(self, options, probabilities):
        """Sample one option safely."""
        probs = np.array(list(probabilities.values()), dtype=float)
        probs = np.clip(probs, 0, None)
        if probs.sum() == 0:
            probs = np.ones(len(options)) / len(options)
        else:
            probs = probs / probs.sum()
        return np.random.choice(options, p=probs)

    def simulate_at_bat(self, pitcher: pd.Series) -> Dict:
        if pitcher is None:
            outcome_probs = {
                'single': 0.15, 'double': 0.05, 'triple': 0.005,
                'homerun': 0.03, 'walk': 0.08, 'strikeout': 0.22,
                'ground_out': 0.24, 'air_out': 0.235, 'hbp': 0.01
            }
            pitcher_name = "Relief Pitcher"
        else:
            pitcher_name = pitcher.get('PlayerName', 'Unknown')
            outcome_probs = self.calculate_pitcher_outcome_probabilities(pitcher)
            if sum(outcome_probs.values()) == 0:
                outcome_probs = {
                    'single': 0.15, 'double': 0.05, 'triple': 0.005,
                    'homerun': 0.03, 'walk': 0.08, 'strikeout': 0.22,
                    'ground_out': 0.24, 'air_out': 0.235, 'hbp': 0.01
                }

        outcome = self.safe_choice(list(outcome_probs.keys()), outcome_probs)

        # Track pitch count
        if self.is_top_inning:
            self.home_pitcher_pitches += random.randint(3, 6)
        else:
            self.away_pitcher_pitches += random.randint(3, 6)

        # Double play chance
        if outcome == 'ground_out' and self.bases[0] and self.outs < 2:
            if random.random() < 0.15:
                outcome = 'double_play'

        return {'outcome': outcome, 'pitcher': pitcher_name}

    def advance_runners(self, outcome: str) -> int:
        runs_scored = 0
        new_bases = [False, False, False]

        if outcome == 'single':
            if self.bases[2]:
                runs_scored += 1
            if self.bases[1]:
                runs_scored += 1 if random.random() < 0.7 else 0
                new_bases[2] = True if random.random() >= 0.7 else new_bases[2]
            if self.bases[0]:
                new_bases[1] = True if random.random() >= 0.6 else new_bases[2]
                new_bases[2] = True if random.random() < 0.6 else new_bases[2]
            new_bases[0] = True

        elif outcome == 'double':
            runs_scored += self.bases[2] + self.bases[1]
            runs_scored += 1 if self.bases[0] and random.random() < 0.8 else 0
            new_bases[1] = True
            if self.bases[0] and random.random() >= 0.8:
                new_bases[2] = True

        elif outcome == 'triple':
            runs_scored += sum(self.bases)
            new_bases[2] = True
        elif outcome == 'homerun':
            runs_scored += sum(self.bases) + 1
        elif outcome in ['walk', 'hbp']:
            if self.bases[0]:
                if self.bases[1]:
                    if self.bases[2]:
                        runs_scored += 1
                    new_bases[2] = True
                new_bases[1] = True
            new_bases[0] = True

        self.bases = new_bases
        return runs_scored

    def process_out(self, outcome: str):
        if outcome == 'double_play':
            self.bases[0] = False
            self.outs += 2
            self.play_by_play.append(f"    DOUBLE PLAY! {self.outs} outs")
        else:
            self.outs += 1
            self.play_by_play.append(f"    {outcome.replace('_',' ').title()}. {self.outs} out(s)")

    def check_pitcher_removal(self):
        if self.is_top_inning:
            if self.home_pitcher_pitches > 100 or self.inning > 6:
                if self.home_pitcher_active:
                    self.home_pitcher_active = False
                    pitcher_name = self.home_pitcher['PlayerName'] if 'PlayerName' in self.home_pitcher else 'Pitcher'
                    self.play_by_play.append(f"    {pitcher_name} removed from game")
        else:
            if self.away_pitcher_pitches > 100 or self.inning > 6:
                if self.away_pitcher_active:
                    self.away_pitcher_active = False
                    pitcher_name = self.away_pitcher['PlayerName'] if 'PlayerName' in self.away_pitcher else 'Pitcher'
                    self.play_by_play.append(f"    {pitcher_name} removed from game")

    def simulate_half_inning(self):
        self.outs = 0
        self.bases = [False, False, False]
        half_inning_runs = 0

        batting_team = self.away_team_name if self.is_top_inning else self.home_team_name
        pitching_team = self.home_team_name if self.is_top_inning else self.away_team_name
        inning_descriptor = "Top" if self.is_top_inning else "Bottom"
        pitcher = self.get_current_pitcher()
        pitcher_name = pitcher['PlayerName'] if pitcher is not None else 'Relief Pitcher'

        self.play_by_play.append(f"\n{'='*60}")
        self.play_by_play.append(f"{inning_descriptor} of Inning {self.inning}")
        self.play_by_play.append(f"{batting_team} batting vs {pitcher_name} ({pitching_team})")
        self.play_by_play.append(f"{'='*60}")

        while self.outs < 3:
            pitcher = self.get_current_pitcher()
            result = self.simulate_at_bat(pitcher)
            outcome = result['outcome']
            self.play_by_play.append(f"\n  At bat vs {result['pitcher']}...")

            if outcome in ['single','double','triple','homerun','walk','hbp']:
                runs = self.advance_runners(outcome)
                half_inning_runs += runs
                if outcome in ['single','double','triple','homerun']:
                    self.play_by_play.append(f"    {outcome.upper()}!")
                    if runs > 0:
                        self.play_by_play.append(f"    {runs} run(s) score!")
                    if self.is_top_inning:
                        self.box_score['away']['H'] += 1
                    else:
                        self.box_score['home']['H'] += 1
                else:
                    self.play_by_play.append(f"    {outcome.upper()}")
                    if runs > 0:
                        self.play_by_play.append(f"    {runs} run(s) score!")
            else:
                self.process_out(outcome)

        if self.is_top_inning:
            self.away_score += half_inning_runs
            self.box_score['away']['R'] += half_inning_runs
        else:
            self.home_score += half_inning_runs
            self.box_score['home']['R'] += half_inning_runs

        self.play_by_play.append(f"\nEnd of half inning. Score: {self.away_team_name} {self.away_score}, {self.home_team_name} {self.home_score}")
        self.check_pitcher_removal()

    def simulate_game(self, innings: int = 9, verbose: bool = True) -> dict:
        for inning in range(1, innings+1):
            self.inning = inning

            self.is_top_inning = True
            self.simulate_half_inning()

            if inning == innings and self.home_score > self.away_score:
                self.play_by_play.append(f"\n{self.home_team_name} wins! No need for bottom of 9th.")
                break

            self.is_top_inning = False
            self.simulate_half_inning()

            if inning >= innings and self.home_score != self.away_score:
                break

        extra_inning = innings + 1
        while self.home_score == self.away_score:
            self.inning = extra_inning
            self.play_by_play.append(f"\n*** EXTRA INNINGS - Inning {extra_inning} ***")
            self.is_top_inning = True
            self.simulate_half_inning()
            self.is_top_inning = False
            self.simulate_half_inning()
            extra_inning += 1
            if extra_inning > innings + 10:
                self.play_by_play.append("\nGame suspended after 19 innings!")
                break

        if self.away_score > self.home_score:
            winner = self.away_team_name
        elif self.home_score > self.away_score:
            winner = self.home_team_name
        else:
            winner = "Tie"

        if verbose:
            for line in self.play_by_play:
                print(line)
            print(f"\n{'='*60}")
            print(f"FINAL SCORE")
            print(f"{'='*60}")
            print(f"{self.away_team_name}: {self.away_score}")
            print(f"{self.home_team_name}: {self.home_score}")
            print(f"\nWinner: {winner}")
            print(f"\nPitching Summary:")
            print(f"{self.home_pitcher.get('PlayerName','Unknown')} ({self.home_team_name}): ~{self.home_pitcher_pitches} pitches")
            print(f"{self.away_pitcher.get('PlayerName','Unknown')} ({self.away_team_name}): ~{self.away_pitcher_pitches} pitches")
            print(f"{'='*60}")

        return {
            'winner': winner,
            'away_score': self.away_score,
            'home_score': self.home_score,
            'innings_played': self.inning,
            'away_hits': self.box_score['away']['H'],
            'home_hits': self.box_score['home']['H'],
            'home_pitcher_pitches': self.home_pitcher_pitches,
            'away_pitcher_pitches': self.away_pitcher_pitches,
            'play_by_play': self.play_by_play
        }


def simulate_multiple_pitcher_games(home_pitcher: pd.Series, away_pitcher: pd.Series,
                                    home_team_name: str = "Home", away_team_name: str = "Away",
                                    num_games: int = 100) -> pd.DataFrame:
    results = []

    print(f"Simulating {num_games} games...")
    print(f"{away_pitcher.get('PlayerName', 'Away Pitcher')} ({away_team_name}) vs {home_pitcher.get('PlayerName', 'Home Pitcher')} ({home_team_name})")

    for game_num in range(num_games):
        
        simulator = PitcherBasedGameSimulator(home_pitcher, away_pitcher, home_team_name, away_team_name)
        result = simulator.simulate_game(verbose=False)
        results.append(result)

    results_df = pd.DataFrame(results)

    print(f"\n{'='*60}")
    print(f"SIMULATION RESULTS - {num_games} Games")
    print(f"{'='*60}")
    print(f"{away_team_name} wins: {sum(results_df['winner'] == away_team_name)} "
          f"({sum(results_df['winner'] == away_team_name)/num_games*100:.1f}%)")
    print(f"{home_team_name} wins: {sum(results_df['winner'] == home_team_name)} "
          f"({sum(results_df['winner'] == home_team_name)/num_games*100:.1f}%)")
    print(f"\nAverage {away_team_name} score: {results_df['away_score'].mean():.2f}")
    print(f"Average {home_team_name} score: {results_df['home_score'].mean():.2f}")
    print(f"\nAverage game length: {results_df['innings_played'].mean():.2f} innings")
    print(f"Extra inning games: {sum(results_df['innings_played'] > 9)}")

    return results_df


In [22]:
away_pitcher = csv_data.loc[csv_data['PlayerName']== "Bryan Woo"].iloc[0]

home_pitcher = csv_data.loc[csv_data['PlayerName']== "Jordan Hicks"].iloc[0]


simulate_multiple_pitcher_games(home_pitcher=home_pitcher, away_pitcher=away_pitcher, 
                                home_team_name="Giants", away_team_name="Mariners", 
                                num_games=1000)


Simulating 1000 games...
Bryan Woo (Mariners) vs Jordan Hicks (Giants)

SIMULATION RESULTS - 1000 Games
Mariners wins: 674 (67.4%)
Giants wins: 326 (32.6%)

Average Mariners score: 5.08
Average Giants score: 3.17

Average game length: 9.18 innings
Extra inning games: 78


,winner,away_score,home_score,innings_played,away_hits,home_hits,home_pitcher_pitches,away_pitcher_pitches,play_by_play
0,Mariners,6,4,10,10,10,197,185,[\n===========================================...
1,Mariners,9,5,9,9,13,200,201,[\n===========================================...
2,Giants,1,6,9,4,10,156,172,[\n===========================================...
3,Mariners,6,2,9,13,4,196,142,[\n===========================================...
4,Mariners,10,4,9,14,8,222,165,[\n===========================================...
...,...,...,...,...,...,...,...,...,...
995,Giants,4,5,9,9,7,175,141,[\n===========================================...
996,Mariners,7,0,9,10,7,206,161,[\n===========================================...
997,Mariners,11,1,9,11,6,203,142,[\n===========================================...
998,Mariners,8,1,9,14,2,208,124,[\n===========================================...
